# Credits

Citation : Ryan Bakker, Liesbet Hooghe, Seth Jolly, Gary Marks, Jonathan Polk, Jan Rovny, Marco Steenbergen, and Milada Anna Vachudova. 2020. “2019 Chapel Hill Expert Survey.” Version 2019.1. Available on chesdata.eu. Chapel Hill, NC: University of North Carolina, Chapel Hill. 


The 2019_CHES_dataset_means.dta Stata file contains average expert judgments per political
party. The 2019_CHES_dataset_expert-level.dta dataset provides information at the level of the
individual expert and allows researchers to aggregate expert scores and estimate standard deviations
among expert judgments

* Completed by : Gayathry Dasika
* Purpose : Fun & Education

In [ ]:
# Libraries
import os
from pickle import dump
from pickle import load

# Dimensionality Reduction
import umap
from sklearn.manifold import TSNE

# Analysis libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

#Visualisation
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
# Jupyter notebook settings
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [ ]:
# This is session dependent. So creating it here.  
try:
    os.mkdir('./data/')
    os.listdir('.')
except:
    pass

# 1: Load data and clean

In [ ]:
# Custom Functions for this section
# Test case to compare two lists
def checkEqual(L1, L2):
    return len(L1) == len(L2) and sorted(L1) == sorted(L2)

In [ ]:
# Read data and load it 
expert_data = pd.read_stata('https://www.chesdata.eu/s/CHES2019_experts.dta')
average_data = pd.read_stata('https://www.chesdata.eu/s/CHES2019V3.dta')

In [ ]:
# Estimate missing values
missing_data = pd.DataFrame(expert_data.isnull().sum().sort_values(ascending=False) / 3823 * 100)
plt.title("Actual Data in the columns")
plt.scatter(np.arange(0,len(expert_data.isnull().sum())),expert_data.isnull().sum().sort_values(ascending=False) / 3823 * 100)
plt.xlabel("Column Number")
plt.ylabel("% information")
plt.show()

In [ ]:
# Check for parties 
parties_expert = expert_data['party_id'].unique()
parties_average = average_data['party_id'].unique()

# There are 277 parties in the survey as per the website. The parties are identified by id
assert len(parties_expert) == 277 
assert len(parties_average) == 277

In [ ]:
# Ideally the average and expert party ids should match. 
try:
    assert checkEqual(parties_expert, parties_average)
except AssertionError:
    print("Assertion Error, Check the 'party_id' codes")

### Minor corrections

In [ ]:
# Column rename 
expert_data = expert_data.rename(columns={'immigra_salience':'immigrate_salience','position':'eu_position'})

In [ ]:
# Replace the 843, 1009 in the expert csv with 844, 1016 (code book reference)
expert_data.loc[(expert_data["party_id"]==843),"party_id"] = 844
expert_data.loc[(expert_data["party_id"]==1009),"party_id"] = 1016

# Run test
checkEqual(expert_data["party_id"].unique(),average_data["party_id"].unique())

### Handling missing values and NaNs

In [ ]:
# The following dictionaries contain column name and unique values in the sample space
# Helps in understanding the structure of data.

all_columns = {}
word_columns = {}
num_columns = {}
dont_know = {} #special case of csv files

for i in expert_data.columns:
    print(i)
    u = expert_data[i].unique()
    print('\nType:\n',set([type(k) for k in u]),'\nValues:\n',u)
    all_columns[i] = u

    if i in ['party_name','cname']:
        word_columns[i] = u
        print(" : Sorted in word_column")
    elif '.d' in u:
        dont_know[i] = u
        print(" : Sorted in dont_know")
    else:
        num_columns[i] = u
        print(" : Sorted in num_column")
    print('=='*50)

### Missing values 

for numerical columns can be done with mean imputation. 
The exceptions to columns are demographics columns = gender, age , etc. Mean imputation doesnt disturb the underlying distribution

In [ ]:
# Create a new df 
df = expert_data.copy()
# Replace the numerical NaN with mean value
for c in num_columns.keys():    
    if c in ['gender','dob']:
        df[c] = df[c].replace(to_replace=np.nan,value=None)
    else:
        df[c] = df[c].fillna(np.mean(df[c]))

### Demographics information 
Removed from analysis

In [ ]:
# Demographics data -  we will drop it for this analysis 
personal_details = df[set(expert_data.columns) - set(average_data.columns)]
personal_attr = personal_details.columns
personal_details["party_id"] = df["party_id"]
personal_details["party"] = df["party"].values
personal_details = personal_details.set_index("party_id")

In [ ]:
# Consider numerical columns only 
df_num = df.drop(columns = personal_details.columns, axis=1)
df_num["party_a_econ"] = personal_details["party_a_econ"].values
df_num["party_b_econ"] = personal_details["party_b_econ"].values
df_num["party_c_econ"] = personal_details["party_c_econ"].values
df_num.head()

## Clean data : Numerical

In [ ]:
df_num.info()

In [ ]:
# Need to scale these items and remove some unwanted standard deviation columns - as they are not votes

u_sd = df_num[["lrecon_sd","eu_position_sd","galtan_sd"]]
df_num = df_num.drop(columns=["lrecon_sd","eu_position_sd","galtan_sd"], axis=1)
del u_sd
df_num.head()

## Store data

In [ ]:
# Load data : Save it locally in serialised format

df.to_pickle('./data/CHES_Expert_clean.pkl')
df_num.to_pickle('./data/CHES_Cleaned_Numerical_data.pkl')
personal_details.to_pickle('./data/CHES_Personal_details.pkl')



# Analysis

In [ ]:
#Viewing the summary stats

df_num = df_num.set_index('party_id') # Primary key
df_num.describe()

## Standard Scaling

In [ ]:
std = StandardScaler()
df_num = df_num.reset_index() # To include the party_id : a combo feature of country and party 
X = std.fit_transform(df_num)
df_scale = pd.DataFrame(index = df_num.index, data=X,columns=df_num.columns)
df_scale.index = df_num["party_id"]
df_scale.to_pickle("Standard_numerical_data.pkl")

# store the pickle file 
dump(std,open('scaler.pkl','wb'))

# 2. Dimensionality Reduction

There are around 53 features. To do a small analysis it is ok to plot the items in 2D and do a feature by feature analysis. The complexity we have in this data are 53 numerical features x 32 countries C 2 - A very large number of graphs that will lead us no where. 

For this purpose, lets attempt to reduce the dimensionality by creating representative 2D features. 

Techniques discussed :T-SNE, UMAP

In [ ]:
df_scale = pd.read_pickle('Standard_numerical_data.pkl')

In [ ]:
# Add identifier columns in the end to the data 
labels = average_data[["country","party","party_id"]]
labels = labels.set_index("party_id")
df_scale = df_scale.join(how='inner',other=labels)
df_scale.head()

# Representing in 2D : t-SNE

The main purpose of t-SNE is visualization of high-dimensional data. Hence, it works best when the data will be embedded on two or three dimensions.

Optimizing the KL divergence can be a little bit tricky sometimes. There are five parameters that control the optimization of t-SNE and therefore possibly the quality of the resulting embedding:

* perplexity
* early exaggeration factor
* learning rate
* maximum number of iterations
* angle (not used in the exact method)

In [ ]:
features = df_scale.loc[:, :'party_c_econ']
tsne = TSNE(n_components=2, random_state=0,n_iter=1000,n_jobs=100,verbose=1,perplexity=30)
projections = tsne.fit_transform(features)

# Data frame for plotting purpose
tsne_df = pd.DataFrame(data=projections,index=df_scale.index,columns=["X1","X2"])
tsne_df["country"] = df_scale["country"].values
tsne_df["party"] = df_scale["party"].values
tsne_df.head()

## Perplexity plots : t-SNE

In [ ]:
# Utility Functions

# Function to run a perplexity plot once with settings 
def run_once_tsne2d(tsne_df=tsne_df,n_iter=1000,n_jobs=100,perplexity=30):
    tsne = TSNE(n_components=2, random_state=0,n_iter=n_iter,n_jobs=n_jobs,verbose=1,perplexity=perplexity)
    p1 = tsne.fit_transform(features)
    print("Perplexity :",perplexity)
    tsne_df["X1_P"+str(perplexity)] = p1[:,0]
    tsne_df["X2_P"+str(perplexity)] = p1[:,1]
    del tsne , p1
    return tsne_df

# Function to run multiple permutations in a series of steps. The points will be appended to the df
def run_permutations_tsne2d(tsne_df=tsne_df,step=5,low=1,high=100,n_iter=1000,n_jobs=100):
    projections = dict()
    for i in range(low,high,step):
        tsne = TSNE(n_components=2, random_state=0,n_iter=n_iter,n_jobs=n_jobs,verbose=1,perplexity=i)
        p1 = tsne.fit_transform(features)
        print("Perplexity :",i)
        projections["Perplexity_"+str(i)] = p1
        del p1, tsne
    for i in range(1,100,5):
        tsne_df["X1_P"+str(i)] = projections['Perplexity_'+str(i)][:,0]
        tsne_df["X2_P"+str(i)] = projections['Perplexity_'+str(i)][:,1]
    return tsne_df


def plot_tsne_1(pp=30,tsne_df=tsne_df.reset_index()):
    # Plots graphs with a perplexity setting
    try:
        x1 = "X1_P"+str(pp)
        y1 = "X2_P"+str(pp)
        fig = px.scatter(tsne_df, 
                         x=x1, 
                         y= y1,
                         color="party",
                         opacity=0.4,
                         title="T-SNE with Perplexity {}".format(pp))
    except:
        x1 = "X1"
        y1 = "X2"
        fig = px.scatter(tsne_df, 
                         x=x1, 
                         y= y1,
                         color="party", 
                         opacity=0.4,
                         title="T-SNE with Perplexity {}".format(pp))
    return fig.show()

In [ ]:
#Select perplexity : 51
p_select = 51
t_df = run_once_tsne2d(tsne_df=tsne_df,perplexity=p_select)
plot_tsne_1(p_select,t_df)

In [ ]:
#Select perplexity : 30
p_select = 30
t_df = run_once_tsne2d(tsne_df=tsne_df,perplexity=p_select)
plot_tsne_1(p_select,t_df)

The main issue here is lack of clear separation. Also for downstream tasks, might need inverse transform. 
Inverse transforms are not supported by this package. Will have to pre-process using PCA and that will disturb the local structure of the data. Also t-SNE is computationally expensive. 

# 2: Representing in 2D: UMAP 
For better visualisation as an improved version of t-SNE, as the clusters are not well defined here. 
There are many settings in UMAP class. Taken the default params for now. 

In [ ]:
df_scale.head()

In [ ]:
# There are many components in UMAP, like t-SNE - Trying the default values here
umap_2d = umap.umap_.UMAP(n_components=2,init='random',random_state=0).fit(df_scale.loc[:,:'party_c_econ'])
projections_2d = umap_2d.transform(df_scale.loc[:,:'party_c_econ'])

In [ ]:
# Create a dataframe to hold the 2d points
umap_df = pd.DataFrame(data=projections_2d,columns=["X1","X2"],index=df_scale.index)
umap_df["party"] = df_scale["party"].values
umap_df["country"] = df_scale["country"].values
umap_df = umap_df.reset_index()

In [ ]:
fig_2d = px.scatter(umap_df, 
                    x="X1", 
                    y="X2",
                    color="party", 
                    labels={'color': 'party'},
                    hover_data=["X1","X2","party","party_id","country"],
                    title="2D UMAP of CHES2019 Data by Parties")

fig_2d_1 = px.scatter(umap_df, 
                    x="X1", 
                    y="X2",
                    color="country", 
                    labels={'color': 'party'},
                    hover_data=["X1","X2","party","party_id","country"],
                    title="2D UMAP of CHES2019 Data - Country shade")

fig_2d.update_traces(marker_size=5)
fig_2d_1.update_traces(marker_size=5)

#Show time
fig_2d.show()
fig_2d_1.show()


In [ ]:
# There are many components in UMAP, like t-SNE - Trying epoch setting
?umap.umap_.UMAP

## UMAP Settings : experiment
Hoping to see a better clustering with more training. 

In [ ]:
# Trying longer epochs - initially was 1000, now 5k
umap_2d_k = umap.umap_.UMAP(n_components=2,
                               n_epochs=5000,
                               verbose=True,
                               init='random',
                               random_state=0).fit(df_scale.loc[:,:'party_c_econ'])
projections_2d_k = umap_2d_k.transform(df_scale.loc[:,:'party_c_econ'])

# Create a dataframe to hold the 2d points
umap_df_k = pd.DataFrame(data=projections_2d_k,columns=["X1","X2"],index=df_scale.index)
umap_df_k["party"] = df_scale["party"].values
umap_df_k["country"] = df_scale["country"].values
umap_df_k = umap_df_k.reset_index()

fig_2d_k = px.scatter(umap_df_k, 
                    x="X1", 
                    y="X2",
                    color="country", 
                    labels={'color': 'party'},
                    hover_data=["X1","X2","party","party_id","country"],
                    title="2D UMAP of CHES2019 Data - Country shade, 1000 epochs")

fig_2d_k.update_traces(marker_size=5)
#Show time
fig_2d_k.show()

In [ ]:
fig_2d_k = px.scatter(umap_df_k, 
                    x="X1", 
                    y="X2",
                    color="party", 
                    labels={'color': 'party'},
                    hover_data=["X1","X2","party","party_id","country"],
                    title="2D UMAP of CHES2019 Data - party shade, 5000 epochs")

fig_2d_k.update_traces(marker_size=5)
#Show time
fig_2d_k.show()

# 3. Sampling 10 New parties
Need to pick up some random 10 parties from this distribution and generate the distribution. This is possible with UMAP Inverse transform. Taking the original default values that dont have negative leaning. 

In [ ]:
# Identify extreme corners 

sample = list()
for i in range(0,10):
    sample.append([np.random.randint(1,11),np.random.randint(2,8)])
print(sample)

inv_transformed_points = umap_2d.inverse_transform(np.array(sample))
random_10 = pd.DataFrame(data=std.inverse_transform(inv_transformed_points),columns=df_scale.loc[:,:'party_c_econ'].columns)
random_10["party_id"] = random_10["party_id"].apply(lambda x: int(x))

In [ ]:
alphabet = map(chr, range(97, 97+len(random_10)))
random_10["party"] = [_.upper()+"(r_10)" for _ in alphabet]
random_10["country_code"] = random_10["party_id"].apply(lambda x: x//100)

# Traceback opinions in high dimension
for i in random_10.columns:
    try:
        random_10[i] = random_10[i].apply(lambda x: int(x))
    except:
        pass

In [ ]:
random_10.columns

In [ ]:
random_10.to_pickle("Generated_parties.pkl")

In [ ]:
random_10

# 5: Brownie points 

The feature values in the high-dimensional data set have finite bounds.(0-10)
In the 2D space you created in Step 2, paint the area that contains exactly the valid /
plausible transformed values corresponding to the high-dimensional feature value bounds.
If this is ill-defined with your chosen dimensionality reduction method, try solving the task
switching your dimensionality reduction method to principal component analysis instead.
Furthermore, in the README, explain why the task was ill-defined with your method.

In [ ]:
umap_df.head()

In [ ]:
px.density_heatmap(umap_df,
    x="X1", 
    y="X2", 
    labels={'color': 'party'},
    hover_data=["X1","X2","party","party_id","country"],
    title="Heatmap of most plausible 2D transformation",
    marginal_x="rug",
    marginal_y="histogram")


# Aggregating

In [ ]:
df_scale_agg = df_scale.groupby(by=df_scale.index).mean()
df_scale_agg["country"] = average_data["country"].values
df_scale_agg["party"] = average_data["party"].values


In [ ]:
# UMAP 
umap_2d_agg = umap.umap_.UMAP(n_components=2,init='random',random_state=0).fit(df_scale_agg.loc[:,:'party_c_econ'])
projections_2d_agg = umap_2d_agg.transform(df_scale_agg.loc[:,:'party_c_econ'])

In [ ]:
# Create a dataframe to hold the 2d points
umap_df_agg = pd.DataFrame(data=projections_2d_agg,columns=["X1","X2"],index=df_scale_agg.index)
umap_df_agg["party"] = df_scale_agg["party"].values
umap_df_agg["country"] = df_scale_agg["country"].values
umap_df_agg = umap_df_agg.reset_index()
umap_df_agg

In [ ]:
fig_2 = px.scatter(
    umap_df_agg, 
    x="X1", 
    y="X2",
    color="country", 
    labels={'color': 'party'},
    hover_data=["X1","X2","party","party_id","country"],
    title="2D UMAP of CHES2019 Data Aggregated Opinions")

fig_2.update_traces(marker_size=10)

#Show time
fig_2.show()

### Thanks for reading. Hope it was fun ! 